<a href="https://colab.research.google.com/github/ImbangJayoGampito/DIF62130_B_25_2311532002/blob/main/Praktikum%204/BernouliNB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,classification_report, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
dataset_dict = {
    'Outlook': ['sunny', 'sunny', 'overcast', 'rainy', 'rainy', 'rainy', 'overcast', 'sunny', 'sunny', 'rainy', 'sunny', 'overcast', 'overcast', 'rainy', 'sunny', 'overcast', 'rainy', 'sunny', 'sunny', 'rainy', 'overcast', 'rainy', 'sunny', 'overcast', 'sunny', 'overcast', 'rainy', 'overcast'],
    'Temperature': [85.0, 80.0, 83.0, 70.0, 68.0, 65.0, 64.0, 72.0, 69.0, 75.0, 75.0, 72.0, 81.0, 71.0, 81.0, 74.0, 76.0, 78.0, 82.0, 67.0, 85.0, 73.0, 88.0, 77.0, 79.0, 80.0, 66.0, 84.0],
    'Humidity': [85.0, 90.0, 78.0, 96.0, 80.0, 70.0, 65.0, 95.0, 70.0, 80.0, 70.0, 90.0, 75.0, 80.0, 88.0, 92.0, 85.0, 75.0, 92.0, 90.0, 85.0, 88.0, 65.0, 70.0, 60.0, 95.0, 70.0, 78.0],
    'Wind': [False, True, False, False, False, True, True, False, False, False, True, True, False, True, True, False, False, True, False, True, True, False, True, False, False, True, False, False],
    'Play': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes']
}
df = pd.DataFrame(dataset_dict)
df.head()

,Outlook,Temperature,Humidity,Wind,Play
0,sunny,85.0,85.0,False,No
1,sunny,80.0,90.0,True,No
2,overcast,83.0,78.0,False,Yes
3,rainy,70.0,96.0,False,Yes
4,rainy,68.0,80.0,False,Yes


In [3]:
df['Wind'] = df['Wind'].astype(int)
df['Play'] = (df['Play'] == "Yes").astype(int)
df.head()

,Outlook,Temperature,Humidity,Wind,Play
0,sunny,85.0,85.0,0,0
1,sunny,80.0,90.0,1,0
2,overcast,83.0,78.0,0,1
3,rainy,70.0,96.0,0,1
4,rainy,68.0,80.0,0,1


In [4]:
df['Temperature'] = pd.cut(df['Temperature'], bins=[0,80,100], labels=['Warm', 'Hot'])
df['Humidity'] = pd.cut(df['Humidity'], bins=[0, 75, 100], labels=['Dry', 'Humid'])
one_hot_columns = pd.get_dummies(df[['Temperature', 'Humidity']], drop_first=True, dtype=int)
df = df.drop(['Temperature', 'Humidity'], axis=1)
df = pd.concat([one_hot_columns, df], axis=1)
df.head()

,Temperature_Hot,Humidity_Humid,Outlook,Wind,Play
0,1,1,sunny,0,0
1,0,1,sunny,1,0
2,1,1,overcast,0,1
3,0,1,rainy,0,1
4,0,1,rainy,0,1


In [5]:
x = df.drop('Play', axis=1)
y = df['Play']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.35, random_state=42)
x_test.head()

,Temperature_Hot,Humidity_Humid,Outlook,Wind
9,0,1,rainy,0
25,0,1,overcast,1
8,0,0,sunny,0
21,0,1,rainy,0
0,1,1,sunny,0


In [32]:
from fractions import Fraction
def naive_bayes(x_train, y_train, x_test, y_test):
    y_data_use = y_train
    likelihoods = {}
    prior = {}
    total_count = y_data_use.value_counts().sum()
    prob_series = y_data_use.value_counts().apply(lambda x: Fraction(x,total_count))
    likelihoods = {}
    for i in range (0, len(prob_series)):
        x_train_cls = x_train[y_train == i]
          # print("class value = ", i, x_train_cls)
        p_y = prob_series[i]
        prior = prob_series
        likelihoods[i] = {}
        for cols in x_train_cls.columns:
             # print("play value = ", i)
             col_value = x_train_cls[cols].value_counts()
             total_in_col = col_value.sum()
             prob_values = col_value.apply(lambda x: Fraction(x,total_in_col))
             likelihoods[i][cols] = prob_values
             # print(prob_values)
        #Print(prob_series)
          # print(likelihoods[cls][i])
            #rint(prob_values)
        # print(f"Class: {cls}")
    #print(prior)
    #print(likelihoods)
    #print(x_test, y_test)
    res = {}
    print(y_test)
    for index, row in x_test.iterrows():
      highest_prob = Fraction(0)
      prediction = 0;
      for i in range(0, len(prior)):
         probability = Fraction(1) * prior[i]
         #print(f"Probability y: {p_y}")
         #print(f"Row index: {index}")
         for col in x_test.columns:
            if row[col] not in likelihoods[i][col]:
                    continue
            chance = likelihoods[i][col][row[col]]
            probability *= chance
         if probability > highest_prob:
            highest_prob = probability
            prediction = i;
      res[index] = prediction
    print(res, y_test)
    return pd.Series(res)
            #print("chance: ", likely)
            #print()
y_pred = naive_bayes(x_train, y_train, x_test, y_test)
y_pred.head()

9     1
25    1
8     1
21    1
0     0
12    1
17    0
22    1
11    1
13    0
Name: Play, dtype: int64
{9: 1, 25: 0, 8: 1, 21: 1, 0: 0, 12: 1, 17: 0, 22: 0, 11: 0, 13: 0} 9     1
25    1
8     1
21    1
0     0
12    1
17    0
22    1
11    1
13    0
Name: Play, dtype: int64


,0
9,1
25,0
8,1
21,1
0,0


In [33]:
# prompt: add accuracy for y_train and y_pred

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.7
